In [1]:
import re
import csv
import time
import requests
import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
match_data = pd.read_csv("./match_data_yearly/all_years.csv")

In [3]:
match_data["Date"] = pd.to_datetime(match_data["Date"])

In [4]:
match_data = match_data.set_index(match_data["Date"])

In [5]:
match_data['2020']

,ID,Year,Sec,Date,Home,Away,HomeID,AwayID,HomeGF,AwayGF,W/L,Stadium,Attendances,HomeElo,AwayElo,HomeED,AwayED
Date,,,,,,,,,,,,,,,,,
2020-02-21,20011203,2020,1,2020-02-21,shonan-bellmare,urawa-red-diamonds,12,3,2,3,2,ＢＭＷス,13071,1422,1484,0,0
2020-02-22,20015408,2020,1,2020-02-22,vegalta-sendai,nagoya-grampus-eight,54,8,1,1,0,ユアスタ,13968,1459,1431,0,0
2020-02-22,20012031,2020,1,2020-02-22,cerezo-osaka,oita-trinita,20,31,1,0,1,ヤンマー,15535,1558,1427,0,0
2020-02-22,20011114,2020,1,2020-02-22,kashiwa-reysol,consadole-sapporo,11,14,4,2,1,三協Ｆ柏,12468,1503,1481,0,0
2020-02-22,20012133,2020,1,2020-02-22,kawasaki-frontale,sagan-tosu,21,33,0,0,0,等々力,21117,1628,1456,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-19,20340120,2020,34,2020-12-19,kashima-antlers,cerezo-osaka,1,20,1,1,0,カシマ,11251,1594,1560,4,-6
2020-12-19,20340810,2020,34,2020-12-19,nagoya-grampus-eight,sanfrecce-hiroshima,8,10,1,0,1,豊田ス,16825,1515,1524,9,-13
2020-12-19,20343331,2020,34,2020-12-19,sagan-tosu,oita-trinita,33,31,2,2,0,駅スタ,8966,1453,1458,17,7


In [6]:
# club_abbies = ["kasm","chib","uraw","tk-v","y-fm","y-fg","shim","nago","g-os","hiro","kasw","shon","iwat","sapp","kobe","c-os","ka-f","fctk","fuku","kyot","omiy","kofu","yama","oita","tosu","y-fc","toku","mats","ngsk","send","niig","y-fm","ka-f","fctk","y-fc"]

In [7]:
# len(club_abbies)

In [8]:
club_id = pd.read_csv("./club_and_id.csv")
club_id

,club_id,club,club_abbr,club_ja
0,1,kashima-antlers,kasm,鹿島
1,2,jef-united,chib,千葉
2,3,urawa-red-diamonds,uraw,浦和
3,4,tokyo-verdy,tk-v,東京Ｖ
4,5,yokohama-fa-marinos,y-fm,横浜FM
5,6,yokohama-flugels,y-fg,横浜Ｆ
6,7,shimizu-s-pulse,shim,清水
7,8,nagoya-grampus-eight,nago,名古屋
8,9,gamba-osaka,g-os,Ｇ大阪
9,10,sanfrecce-hiroshima,hiro,広島


In [9]:
# club_id.insert(2,"club_abbr",club_abbies)

In [11]:
# for index,row in match_data[f'{year}'].iterrows():
#     month = row["Date"].month
#     date = row["Date"].day
    
#     home=row["Home"]
#     away=row["Away"]
    
#     match_id=row["ID"]
    
#     for i, r in club_id.iterrows():
#         if home == r["club"]:
#             club_abbr = r["club_abbr"]
            
#     print(match_id,home,away,club_abbr)
        
        

In [12]:
year = '2020'

In [13]:
month = '12'
date = '01'

In [14]:
club_abbr = 'send'

In [15]:
match_id = '20265411'

In [16]:
home='vegalta-sendai'
away='kashiwa-reysol'

In [17]:
url = f'https://www.football-lab.jp/{club_abbr}/report/?year={year}&month={month}&date={date}'
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

In [18]:
soup

<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="ja" http-equiv="Content-Language"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="text/css" http-equiv="Content-Style-Type"/>
<meta content="text/javascript" http-equiv="Content-Script-Type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="index,follow" name="robots"><title>ベガルタ仙台 2020マッチレポート | 12月1日 vs 柏 | データによってサッカーはもっと輝く | Football LAB</title>
<meta content="サッカー,Jリーグ,データ,ランキング,フットボールラボ,Football LAB,チャンスビルディングポイント,CBP,J1,J2,J3,ベガルタ仙台,マッチレポート,ゲームレポート,レポート,試合" name="keywords"/>
<meta content="フットボールラボ(Football LAB)はサッカーをデータで分析し、新しいサッカーの観戦方法を伝えるサッカー情報サイトです。選手のプレーを評価するチャンスビルディングポイントやプレースタイル指標、チームの戦術を評価するチームスタイル指標といった独自のデータを開発しています。データを活用してサッカーに新しい視点を提供するとともに、レポートやコラムを通してJリーグの試合を分析・解説します。" name="description"/>
<link href="https://www

In [19]:
records = soup.select(f"#{club_abbr} > article > div:nth-child(10) > table:nth-child(44) tr")
records

[<tr>
 <th class="dsktp">今季平均</th>
 <th class="">成功率</th>
 <th class="">総数</th>
 <th class="dsktp"></th>
 <th class="">総数</th>
 <th class="">成功率</th>
 <th class="dsktp">今季平均</th>
 </tr>,
 <tr class="tr3 sp"><td colspan="4">ゴール期待値</td></tr>,
 <tr class="tr2">
 <td class="dsktp numNA c">1.194</td>
 <td class="c numNA">-</td>
 <td class="c">1.137</td>
 <td class="dsktp c">ゴール期待値</td>
 <td class="c num1st">1.955</td>
 <td class="c numNA">-</td>
 <td class="dsktp numNA c">1.354</td>
 </tr>,
 <tr class="tr3 sp"><td colspan="4">シュート</td></tr>,
 <tr>
 <td class="dsktp numNA c">11.2</td>
 <td class="c">(0.0<span class="s">%</span>)</td>
 <td class="wi2c c">8</td>
 <td class="dsktp c">シュート</td>
 <td class="wi2c c num1st">13</td>
 <td class="c num1st">(15.4<span class="s">%</span>)</td>
 <td class="dsktp numNA c">13.6</td>
 </tr>,
 <tr class="tr3 sp"><td colspan="4">枠内シュート</td></tr>,
 <tr class="tr2">
 <td class="dsktp numNA c">3.5</td>
 <td class="c numNA">-</td>
 <td class="c">1</td>
 <td class

In [20]:
del records[1::2]

In [21]:
records

[<tr>
 <th class="dsktp">今季平均</th>
 <th class="">成功率</th>
 <th class="">総数</th>
 <th class="dsktp"></th>
 <th class="">総数</th>
 <th class="">成功率</th>
 <th class="dsktp">今季平均</th>
 </tr>,
 <tr class="tr2">
 <td class="dsktp numNA c">1.194</td>
 <td class="c numNA">-</td>
 <td class="c">1.137</td>
 <td class="dsktp c">ゴール期待値</td>
 <td class="c num1st">1.955</td>
 <td class="c numNA">-</td>
 <td class="dsktp numNA c">1.354</td>
 </tr>,
 <tr>
 <td class="dsktp numNA c">11.2</td>
 <td class="c">(0.0<span class="s">%</span>)</td>
 <td class="wi2c c">8</td>
 <td class="dsktp c">シュート</td>
 <td class="wi2c c num1st">13</td>
 <td class="c num1st">(15.4<span class="s">%</span>)</td>
 <td class="dsktp numNA c">13.6</td>
 </tr>,
 <tr class="tr2">
 <td class="dsktp numNA c">3.5</td>
 <td class="c numNA">-</td>
 <td class="c">1</td>
 <td class="dsktp c">枠内シュート</td>
 <td class="c num1st">4</td>
 <td class="c numNA">-</td>
 <td class="dsktp numNA c">4.7</td>
 </tr>,
 <tr>
 <td class="dsktp numNA c">0.1

In [22]:
data = []

for record in records:
    l = [field.text for field in record.find_all(['th','td'])]           
    data.append(l)

In [23]:
data

[['今季平均', '成功率', '総数', '', '総数', '成功率', '今季平均'],
 ['1.194', '-', '1.137', 'ゴール期待値', '1.955', '-', '1.354'],
 ['11.2', '(0.0%)', '8', 'シュート', '13', '(15.4%)', '13.6'],
 ['3.5', '-', '1', '枠内シュート', '4', '-', '4.7'],
 ['0.1', '-', '0', 'PKによるシュート', '0', '-', '0.1'],
 ['409.6', '(77.0%)', '570', 'パス', '548', '(80.1%)', '437.9'],
 ['17.3', '(14.8%)', '27', 'クロス', '20', '(20.0%)', '14.6'],
 ['9.4', '-', '13', '直接ＦＫ', '14', '-', '10.9'],
 ['1.9', '-', '1', '間接ＦＫ', '0', '-', '2.4'],
 ['4.4', '-', '5', 'ＣＫ', '3', '-', '4.2'],
 ['21.7', '(94.1%)', '17', 'スローイン', '18', '(94.4%)', '21.5'],
 ['10.8', '(58.3%)', '12', 'ドリブル', '6', '(66.7%)', '8.6'],
 ['16.9', '(81.8%)', '11', 'タックル', '22', '(77.3%)', '21.7'],
 ['23.3', '-', '12', 'クリア', '29', '-', '25.7'],
 ['2.0', '-', '1', 'インターセプト', '4', '-', '2.0'],
 ['1.8', '-', '0', 'オフサイド', '1', '-', '2.1'],
 ['1.2', '-', '0', '警告', '0', '-', '1.6'],
 ['0.0', '-', '0', '退場', '0', '-', '0.1'],
 ['33.4', '-', '53', '３０ｍライン進入', '46', '-', '38.5'],
 ['9.4', '-', 

In [24]:
# with open(f'./stats/{year}/{match_id}.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerows(data)

In [25]:
df = pd.read_csv(f'./stats/{year}/{match_id}.csv', index_col=3)

In [26]:
df

,今季平均,成功率,総数,総数.1,成功率.1,今季平均.1
ゴール期待値,1.194,-,1.137,1.955,-,1.354
シュート,11.2,(0.0%),8,13,(15.4%),13.6
枠内シュート,3.5,-,1,4,-,4.7
PKによるシュート,0.1,-,0,0,-,0.1
パス,409.6,(77.0%),570,548,(80.1%),437.9
クロス,17.3,(14.8%),27,20,(20.0%),14.6
直接ＦＫ,9.4,-,13,14,-,10.9
間接ＦＫ,1.9,-,1,0,-,2.4
ＣＫ,4.4,-,5,3,-,4.2
スローイン,21.7,(94.1%),17,18,(94.4%),21.5


In [27]:
df = df.drop({"今季平均","成功率","成功率.1","今季平均.1"}, axis=1)

In [28]:
df

,総数,総数.1
ゴール期待値,1.137,1.955
シュート,8,13
枠内シュート,1,4
PKによるシュート,0,0
パス,570,548
クロス,27,20
直接ＦＫ,13,14
間接ＦＫ,1,0
ＣＫ,5,3
スローイン,17,18


In [29]:
df = df.rename(columns={'総数':f'{home}', '総数.1':f'{away}'})

In [30]:
df

,vegalta-sendai,kashiwa-reysol
ゴール期待値,1.137,1.955
シュート,8,13
枠内シュート,1,4
PKによるシュート,0,0
パス,570,548
クロス,27,20
直接ＦＫ,13,14
間接ＦＫ,1,0
ＣＫ,5,3
スローイン,17,18


In [34]:
new_index = []
if len(df.index) == 24:
    new_index = ["ExpG","Shots","ShotsOnTarget","ShotsFromPK","Passes","Crosses","DirectFK","IndirectFK","CK","Throwin","Dribbling","Tackles","Clearances","Intercepts","Offsides","YellowCards","RedCards","30mLineEntries","PenaltyAreaEntries","DistanceCovered","Sprints","Attacks","ChanceBuildingRate","Possession"]
elif len(df.index) == 23:
    new_index = ["Shots","ShotsOnTarget","ShotsFromPK","Passes","Crosses","DirectFK","IndirectFK","CK","Throwin","Dribbling","Tackles","Clearances","Intercepts","Offsides","YellowCards","RedCards","30mLineEntries","PenaltyAreaEntries","DistanceCovered","Sprints","Attacks","ChanceBuildingRate","Possession"]
else :
    print(match_id)

In [35]:
new_index

['ExpG',
 'Shots',
 'ShotsOnTarget',
 'ShotsFromPK',
 'Passes',
 'Crosses',
 'DirectFK',
 'IndirectFK',
 'CK',
 'Throwin',
 'Dribbling',
 'Tackles',
 'Clearances',
 'Intercepts',
 'Offsides',
 'YellowCards',
 'RedCards',
 '30mLineEntries',
 'PenaltyAreaEntries',
 'DistanceCovered',
 'Sprints',
 'Attacks',
 'ChanceBuildingRate',
 'Possession']

In [36]:
df[""] = new_index

In [37]:
df = df.set_index("")

In [38]:
df

,vegalta-sendai,kashiwa-reysol
,,
ExpG,1.137,1.955
Shots,8,13
ShotsOnTarget,1,4
ShotsFromPK,0,0
Passes,570,548
Crosses,27,20
DirectFK,13,14
IndirectFK,1,0
CK,5,3


In [ ]:
df.at["DistanceCovered",f"{home}"] = df.at["DistanceCovered",f"{home}"].replace("m","").replace(",","")

In [ ]:
df.at["DistanceCovered",f"{away}"] = df.at["DistanceCovered",f"{away}"].replace("m","").replace(",","")

In [ ]:
df.at["ChanceBuildingRate",f"{home}"] = df.at["ChanceBuildingRate",f"{home}"].replace("%","")
df.at["ChanceBuildingRate",f"{away}"] = df.at["ChanceBuildingRate",f"{away}"].replace("%","")

In [ ]:
df.at["Possession",f"{home}"] = df.at["Possession",f"{home}"].replace("%","")
df.at["Possession",f"{away}"] = df.at["Possession",f"{away}"].replace("%","")

In [ ]:
df

In [ ]:
df.to_csv(f"./stats/{year}/{match_id}.csv")

In [ ]:
pd.read_csv(f"./stats/{year}/{match_id}.csv", index_col=0)

### ここから

In [ ]:
import re
import csv
import time
import requests
import datetime
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
club_id = pd.read_csv("./club_and_id.csv")
club_id

In [ ]:
match_data = pd.read_csv("./match_data_yearly/all_years.csv")
match_data["Date"] = pd.to_datetime(match_data["Date"])
match_data = match_data.set_index(match_data["Date"])

In [ ]:
year=2015

In [ ]:
for index,row in match_data[f'{year}'].head(10).iterrows():
    month = row["Date"].month
    date = row["Date"].day

    home=row["Home"]
    away=row["Away"]

    match_id=row["ID"]

    for i, r in club_id.iterrows():
        if home == r["club"]:
            club_abbr = r["club_abbr"]

    url = f'https://www.football-lab.jp/{club_abbr}/report/?year={year}&month={month:02}&date={date:02}'
    r = requests.get(url)
    
    if r.status_code != 200:
        print(match_id)
        if home == 'fc-tokyo':
            url = f'https://www.football-lab.jp/f-tk/report/?year={year}&month={month:02}&date={date:02}'    
            r = requests.get(url)
            
    soup = BeautifulSoup(r.text, 'html.parser')

    table = soup.select(f"article  table.statsTbl6")
    records = table[1].select("tr")
    
    del records[1::2]
    

    data = []
    for record in records:
        l = [field.text for field in record.find_all(['th','td'])]           
        data.append(l)


#     with open(f'./stats/{year}/{match_id}.csv', 'w') as f:
#         writer = csv.writer(f)
#         writer.writerows(data)


    time.sleep(1)

In [ ]:
pd.read_csv(f'./stats/{year}/{match_id}.csv')

In [ ]:
data

In [ ]:
df = pd.DataFrame(data)

In [ ]:
df

In [ ]:
df.columns = ["今季平均","成功率",'総数', '','総数.1',"成功率.1","今季平均.1"]

In [ ]:
df

In [ ]:
df = df.drop(df.index[0])

In [ ]:
df = df.set_index("")
df

In [ ]:
df = df.drop({"今季平均","成功率","成功率.1","今季平均.1"}, axis=1)
df = df.rename(columns={'総数':f'{home}', '総数.1':f'{away}'})

In [ ]:
df

In [ ]:
new_index = []

In [ ]:
if len(df.index) == 24:
    new_index = ["ExpG","Shots","ShotsOnTarget","ShotsFromPK","Passes","Crosses","DirectFK","IndirectFK","CK","Throwin","Dribbling","Tackles","Clearances","Intercepts","Offsides","YellowCards","RedCards","30mLineEntries","PenaltyAreaEntries","DistanceCovered","Sprints","Attacks","ChanceBuildingRate","Possession"]
elif len(df.index) == 23:
    new_index = ["Shots","ShotsOnTarget","ShotsFromPK","Passes","Crosses","DirectFK","IndirectFK","CK","Throwin","Dribbling","Tackles","Clearances","Intercepts","Offsides","YellowCards","RedCards","30mLineEntries","PenaltyAreaEntries","DistanceCovered","Sprints","Attacks","ChanceBuildingRate","Possession"]

In [ ]:
df[""] = new_index
df = df.set_index("")
df

In [ ]:
df

In [ ]:
df.at["DistanceCovered",f"{home}"] = df.at["DistanceCovered",f"{home}"].replace("m","").replace(",","")
df.at["DistanceCovered",f"{away}"] = df.at["DistanceCovered",f"{away}"].replace("m","").replace(",","")

In [ ]:
df

In [ ]:
df.at["ChanceBuildingRate",f"{home}"] = df.at["ChanceBuildingRate",f"{home}"].replace("%","")
df.at["ChanceBuildingRate",f"{away}"] = df.at["ChanceBuildingRate",f"{away}"].replace("%","")

In [ ]:
df.at["Possession",f"{home}"] = df.at["Possession",f"{home}"].replace("%","")
df.at["Possession",f"{away}"] = df.at["Possession",f"{away}"].replace("%","")

In [ ]:
df

In [ ]:
# df.to_csv(f"./stats/{year}/{match_id}.csv")